In [37]:
from bs4 import BeautifulSoup
import requests
import os
import re
import pandas as pd

In [2]:
#try to create a new folder 'data'
try:
    os.mkdir('data')
    print("directory created")
except Exception as e: #'e' means for the same instruction - in this case, the directory 'data'
    print(f"Directory already exists {e}")
    pass

Directory already exists [WinError 183] Cannot create a file when that file already exists: 'data'


In [38]:
def save(t,n):
    #input the job page content as soup object t
    #convert t to string s
    s=t.prettify()
    
    #save the job page content in a text file number n as string
    with open(f"data/{n}.txt","w",encoding='utf-8') as g:
        g.write(s)
    return

In [39]:
def scrape(x,y):
    #to scrape the pages from page x to page y where y>=x.
    #it returns a list of soup objects of job page content
    
    #get the link
    URL = "https://hk.jobsdb.com/hk/search-jobs/data-scientist/"
    
    for p in range(x,y+1):
        #to make valid URL e.g https://hk.jobsdb.com/hk/search-jobs/data-science/1
        nextlink = URL+str(p)
        
        #if the link exists,get the resources
        if nextlink:
            html = requests.get(nextlink)
            soup = BeautifulSoup(html.text, "html.parser")

            #to find the links(directing to the job pages) from the resources
            #list l stores all the links
            l=[]
            for i in soup.find_all('a', href=True):
                if ('hk/en/job' in i['href'])&(i['href'][0]=="/"):
                    l.append("https://hk.jobsdb.com"+i['href'])

            #go to each job page by each link and get the content
            #result (list) stores the job contents
            result=[]
            for k in range(len(l)):
                i=l[k]
                #print("going to",i) #for checking only
                subhtml = requests.get(i)
                #print("success or not?",subhtml) #for checking only
                jobcontent = BeautifulSoup(subhtml.text, "html.parser")
                result.append(jobcontent)
                save(jobcontent,(p-1)*30+k+1)
                
        
        #if the link does not exist, it breaks the for loop.
        #i found that if the link does not exist, the DB website will
          #redirect you to page 1 automatically, which makes this part of code
          #does not work.
        else:
            print("Page",p,"does not exist.")
            break
    return result

In [40]:
#scrape the pages from 133 to 171
scrape(133,171)

section I: scrapping and saving as text
_______________________________________________
section II: gathering data

In [41]:
#to read the job content.txt:
def read(casenum):
    with open(f"data/{casenum}.txt","r",encoding='utf-8') as onlyread:
        text=onlyread.read()
    with open(f"data/{casenum}.txt","r",encoding='utf-8') as onlyread:
        line=onlyread.readlines()
    soup = BeautifulSoup(text, "html.parser")
    return text,line,soup

In [42]:
#transferred
def wordlocation(text,line,word,min_line,max_line):
    #it returns all the non-coding sentence(str) in a list after the target word
    #if max=min, a string is returned (but not a list)
    
    #text,line: pass variable text and line to the function
    #word: input the target word
    #min/max: the range of the non-coding sentence after the target word
    
    s=str(re.search(word,text))
    l=re.findall('\d+', s)
    if l==[]:
        #print("the word does not exist, return none") #for checking only
        return
    g=text[:int(l[1])+1].count("\n")

    p=1
    result=[]
    for j in range(g+1,g+60):
        if line[j].strip()=='':
            continue
        
        if line[j].strip()[0]!="<":
            if ("Additional Information" in line[j].strip())==True:
                #print("already in add. info part") #for checking only
                if p==1:
                    return
                else:
                    if max_line==min_line:
                        return
                    else:
                        return result
            
            if line[j].strip()=="N/A":
                #print("they wrote N/A") #for checking only
                if p==1:
                    return
                else:
                    if max_line==min_line:
                        return
                    else:
                        return result
            
            if (p<=max_line)&(p>=min_line)&(len(line[j].strip())<800):
                if max_line==min_line:
                    return line[j].strip()
                else:
                    result.append(line[j].strip())
                    if p==max_line:
                        return result    
            p+=1

    #print("end of search. return none if nth is found. return a list if sth is found.") #for checking only
    if max_line==min_line:
        return
    else:
        if result==[]:
            return
        else:
            return result

In [43]:
#find post date. It returns date(str) in dd-mmm-yy (date,month,year format)
def date(text):
    s=str(re.search("Posted on",text))
    if s==None:
        s=str(re.search("Posted",text))
        if s==None:
            return
        else:
            return "4-Nov-21"
    else:
        datelist=re.findall('\d+', s)
        y=text[int(datelist[1])+1:int(datelist[1])+10].strip()
        if text.strip()[0].isdigit()==False:
            s=str(re.search("Posted",text))
            if s==None:
                return
            else:
                return "04-Nov-21"
        
        return y

In [44]:
#find job title. It returns a string
def job(soup):
    return soup.find('h1').text.strip()

In [45]:
#it returns company name in string
def company(soup):
    longclass="sx2jih0 zcydq82q _18qlyvc0 _18qlyvcv _18qlyvc2 _1d0g9qk4 _18qlyvc9"
    return soup.find('span',class_=longclass).find_next('span',class_=longclass).text.strip()

In [46]:
#find location in string
def location(soup):
    y = soup.find('div',class_="sx2jih0 zcydq832").text
    if y==None:
        return
    else:
        return y.strip()

In [47]:
#find industry in string
def industry(text,line):
    s=str(re.search("Additional Company Information\n",text))
    l=re.findall('\d+', s)
    g=text[:int(l[1])+1].count("\n")
    
    p=0
    for j in range(g,g+20):
        if line[j].strip()[0]!="<":
            p+=1
        if p==2:
            return line[j].strip()
    #print("no industry found. return none") #for checking only
    return

In [48]:
#find years of experience
def exp(text,line):
    return wordlocation(text,line,"Years of Experience\n",1,1)

In [49]:
#find requirements in list
def req(text,line):
    word=["Requirements:","Requirements","Requirement","requirements:","requirements","requirement","REQUIREMENT"]
    
    for j in word:
        k = wordlocation(text,line,j,1,20)
        if (k!=[])|(k!=None):
            return k
    return

In [50]:
#find responsibilities in list
def res(text,line):
    word=["Responsibilities:","Responsibilities","responsibilities:","responsibilities","responsibility","Responsibility","RESPONSIBILIT"]
    
    for j in word:
        k = wordlocation(text,line,j,1,20)
        if (k!=[])|(k!=None):
            return k
    return

In [51]:
#find job description
def descri(text,line):
    return wordlocation(text,line,"Job Description",1,80)

In [52]:
#find the job case information,initial_casenum<=final_casenum
def jobinfo(initial_casenum,final_casenum):

    for i in range(initial_casenum,final_casenum+1):
        text,line,soup=read(i)

        print("job case:",i)
        print("date:",date(text))
        print("job title:",job(soup))
        print("company name:",company(soup))
        print("location:",location(soup))
        print("industry:",industry(text,line))
        print("yrs of exp:",exp(text,line))
        print("requirements=>",req(text,line),"\n")
        print("responsibilities=>",res(text,line),"\n")
        print("job description=>",descri(text,line),"\n")
        print("______________________The end________________________")
    return

In [18]:
###summary of functions:
#read(casenum): return text,line,soup
#date(text): return str(dd-mmm-yy) / none
#job(soup): return string
#company(soup): return string
#location(soup): return string / none
#industry(text,line): return string / none
#exp(text,line): return string / none
#req(text,line): return string / list / none
#res(text,line): return string / list / none
#descri(text,line): return list / none
#jobinfo(initial_casenum,final_casenum): print all the cases. return none.

section II: gathering data
_________________________________________
section III: finding abnormal job case

******************please read section A: appendix************************
In some cases we could not find any requirements. We figured out the reasons.

section III: finding abnormal job case
______________________________________________________
section IV: data frame

In [53]:
#it reads the job contents and returns a dataframe
def ourdf(initial_casenumber,final_casenumber):
    try:
        read(final_casenumber)
    except Exception as e: #'e' means for the same instruction - in this case, the directory 'data'
        print(f"the file {final_casenumber} doesn't exist ah bro~~ {e}")
        pass
    
    col=["date of post","job title","company name","location","industry","years of experience","requirement","responsibility","job description"]
    df = pd.DataFrame(columns=col,index=range(initial_casenumber,final_casenumber+1))

    for i in range(initial_casenumber,final_casenumber+1):
        text,line,soup=read(i)

        df.loc[i,"date of post"]=date(text)
        df.loc[i,"job title"]=job(soup)
        df.loc[i,"company name"]=company(soup)
        df.loc[i,"location"]=location(soup)
        df.loc[i,"industry"]=industry(text,line)
        df.loc[i,"years of experience"]=exp(text,line)
        df.loc[i,"requirement"]=req(text,line)
        df.loc[i,"responsibility"]=res(text,line)
        df.loc[i,"job description"]=descri(text,line)
        
        print("case number",i,": done")
    return df

In [20]:
df_kent = ourdf(3871,5130)
df_kent.to_csv(path_or_buf="df_kent.csv")

#Each groupmate scraped different set of data and generated their own dataframe
#We combined the dataframe in the next section (section B:clean)

In [54]:
df269 = ourdf(269,269)
df269.to_csv(path_or_buf="df269.csv")
print("csv file saved!")

case number 269 : done
csv file saved!


In [ ]:
#___________________________all the codes below are miscellaneous______
jobinfo(4136,4136)

In [33]:
text,line,soup=read(4136)
date(text)

'04-Nov-21'

In [29]:
text,line,soup=read(269)
jobinfo(269,269)

job case: 269
date: 04-Nov-21
job title: Customer Service/Business Analyst
company name: International Transport Information Systems Limited
location: Kowloon Bay
industry: Information Technology
yrs of exp: 1 year
requirements=> None 

responsibilities=> None 

job description=> ['Job Description:', '- Support customers in daily usage of ITIS software packages through phone call and email', '- Gather preliminary information and pass to second line support', '- Understand new requests from customers on ITIS software packages', '- Conduct user testing on ITIS software packages', '- Provide user training on ITIS software packages', '- Prepare and maintain user documentation, such as user manual,training manual, etc.', 'Job Requirement:', '- With 1 year or above relevant experience;', '- Knowledge in logistics, freight forwarding, and transportation is a plus;', '- Responsive, good communication skill, highly adaptable and open to new tasks/ exposure;', '- Hands-on word processing skill i

In [30]:
descri(text,line)

['Job Description:',
 '- Support customers in daily usage of ITIS software packages through phone call and email',
 '- Gather preliminary information and pass to second line support',
 '- Understand new requests from customers on ITIS software packages',
 '- Conduct user testing on ITIS software packages',
 '- Provide user training on ITIS software packages',
 '- Prepare and maintain user documentation, such as user manual,training manual, etc.',
 'Job Requirement:',
 '- With 1 year or above relevant experience;',
 '- Knowledge in logistics, freight forwarding, and transportation is a plus;',
 '- Responsive, good communication skill, highly adaptable and open to new tasks/ exposure;',
 '- Hands-on word processing skill in English and/or Chinese;',
 '- Good command of spoken &amp; written English and/or Chinese']

1
y 2
3
